In [3]:
%matplotlib widget

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import pickle

import themachinethatgoesping as Ping

In [4]:
!pwd

/ssd/src/themachinethatgoesping/tutorials


In [2]:
files = Ping.echosounders.index_functions.find_files('.','.all')
files = Ping.echosounders.index_functions.find_files(
    '/home/users/data/no_backup/GEOMAR/Meteor-M143/MBES/M143_EM710_raw/',['.all','.wcd']
)
files = pickle.load(open('files.pcl','rb'))

#files = Ping.echosounders.index_functions.find_files(
#    '/home/data/test_data/Egmont_Key/dotall/',['.all','.wcd']
#)

# files = Ping.echosounders.index_functions.find_files(
#     '/home/data/turbeams/TURBEAMS_data_crunshing/campaigns/TURBEAMS_December_2023_25/',['.all','.wcd']
# )

    
# files = Ping.echosounders.index_functions.find_files(
#     '/home/ssd/src/themachinethatgoesping/tutorials/unittest_data/',['.all','.wcd']
# )
# files = Ping.echosounders.index_functions.find_files(
#     '/home/ssd/src/themachinethatgoesping/tutorials/unittest_data/',['.wcd']
# )

                                                    
cache_files = Ping.echosounders.index_functions.get_cache_file_paths(files)

fh = Ping.echosounders.kongsbergall.KongsbergAllFileHandler(files,cache_files)

Found 23 files + 1 duplicates (discarded)
Found 435 files


FileNotFoundError: [Errno 2] No such file or directory: 'files.pcl'

In [ ]:
print(fh)

In [ ]:
pings = fh.get_pings()
len(pings)

In [ ]:
all_pings = Ping.pingprocessing.filter_pings.by_features(fh.get_pings(),['watercolumn.amplitudes'])
pings_per_survey = Ping.pingprocessing.split_pings.by_time_difference(all_pings,10*60)

for k, p in pings_per_survey.items():
    print(k,len(p))                                                        

In [ ]:
fig_nav, ax_nav = Ping.pingprocessing.overview.nav_plot.create_figure(
    'Surveys',                                                                      
    #background_image_path='/home/users/data/no_backup/GEOMAR/Meteor-M143/gis/flaremap_export/indication/-77_5_individual-indication-supermap-downsampled-3-1-center-sector-1mxy-1mz-downsampled-3-[4].tiff',

)

for k, pings in pings_per_survey.items():
    overview = Ping.pingprocessing.overview.get_ping_overview(pings)
    overview.plot_navigation(ax_nav, label=k)

ax_nav.legend()

In [ ]:
from tqdm.auto import tqdm
for ping in tqdm(all_pings):
    pass

In [ ]:
ping.watercolumn.get_amplitudes()

In [ ]:
pings = pings_per_survey[5]
pings = all_pings

pss = Ping.echosounders.pingtools.PingSampleSelector()
pss.select_beam_range_by_angles(-3,3,1)
echodata = Ping.pingprocessing.watercolumn.echograms.EchogramBuilder.from_pings(pings,pss=pss)

In [ ]:
ping.get_geolocation().print()

In [ ]:
from IPython.display import display
import ipywidgets as widgets

viewer = Ping.pingprocessing.widgets.WCIViewer(pings, show=False)

output = widgets.Output()
update_button = widgets.Button(description="update")
clear_button = widgets.Button(description="clear output")

max_x_steps = 2048
max_y_steps = 512
date_time = True

if date_time:
    echodata.set_x_axis_date_time(max_steps=max_x_steps)
else:
    echodata.set_x_axis_ping_nr(max_steps=max_x_steps)
echodata.set_y_axis_depth(max_samples=max_y_steps)

image_background, extent_background = echodata.build_image()

plt.ioff()
fig_eg,ax_eg = Ping.pingprocessing.core.create_figure('echogram')
#fig_eg.set_size_inches(10,5)
plt.ion()


if ('high_res_image' in globals()) and ('high_res_extent' in globals()) and ('high_res_images' in globals()):
    del high_res_image, high_res_extent, high_res_images

show_nav = True
def set_nav(event):
    if not show_nav:
        return
    global latlon
    if 'latlon' in globals():
        latlon.remove()
    geo = viewer.imagebuilder.pings[viewer.w_index.value].get_geolocation()
    latlon = ax_nav.scatter(geo.longitude, geo.latitude,c='black',s=300,zorder=1000)

def update_background(event):
    with output:
        print(viewer.w_vmin.value,viewer.w_vmax.value,)
        global background, high_res_images
        if 'background' in globals():
            background.remove()
        background = ax_eg.imshow(image_background.transpose(), extent=extent_background, cmap = 'YlGnBu_r', vmin = viewer.w_vmin.value, vmax = viewer.w_vmax.value, aspect='auto')
        fig_eg.canvas.flush_events()

        if ('high_res_image' in globals()) and ('high_res_extent' in globals()) and ('high_res_images' in globals()):
            high_res_images.append(ax_eg.imshow(high_res_image.transpose(), extent=high_res_extent, cmap = 'YlGnBu_r', vmin = viewer.w_vmin.value, vmax = viewer.w_vmax.value, aspect='auto'))
            if len(high_res_images) > 3:
                for im in high_res_images[:-3]:
                    im.remove()
                high_res_images = high_res_images[-3:]

update_background(0)

viewer.w_vmin.on_trait_change(update_background)
viewer.w_vmax.on_trait_change(update_background)
viewer.w_index.on_trait_change(set_nav)

def update_ping_line(event):
    with output:
        global ping_line
        if 'ping_line' in globals():
            try:
                ping_line.remove()
            except:
                pass
        ping_line = ax_eg.axvline(viewer.imagebuilder.pings[viewer.w_index.value].get_datetime(),c='black',linestyle='dashed')


viewer.w_index.on_trait_change(update_ping_line)

if date_time:
    Ping.pingprocessing.core.set_ax_timeformat(ax_eg)

viewer.button_layout = widgets.VBox(children = viewer.layout.children[1:])


display(
    widgets.HBox(children=[viewer.fig.canvas, fig_nav.canvas]),
    widgets.HBox(children=[fig_eg.canvas]),
    widgets.HBox(children=[viewer.button_layout]),
    widgets.HBox(children=[update_button, clear_button]), 
    output
)

In [ ]:
from matplotlib import dates as mdates

if 'fig_events' in globals():
    for fig, e in fig_events.values():
       fig_eg.canvas.mpl_disconnect(e)
if 'high_res_images' in globals():
    for e in high_res_images:
        e.remove()
        
fig_events = {}
high_res_images = []

def clear_output(event):
    with output:
        output.clear_output()
def update_echogram(event):
    with output:
        global high_res_images, high_res_image, high_res_extent
        print('update')
        
        xmin,xmax = ax_eg.get_xlim()
        ymin,ymax = sorted(ax_eg.get_ylim())
        
        if date_time:
            tmin,tmax = mdates.num2date(xmin).timestamp(),mdates.num2date(xmax).timestamp()
            echodata.set_x_axis_date_time(max_steps=max_x_steps, min_ping_time=tmin, max_ping_time=tmax)
        else:
            echodata.set_x_axis_ping_nr(max_steps=max_x_steps, min_ping_nr=xmin, max_ping_nr=xmax)
        echodata.set_y_axis_depth(max_samples=max_y_steps, min_depth=ymin, max_depth=ymax)
        
        high_res_image, high_res_extent = echodata.build_image()
        high_res_images.append(ax_eg.imshow(high_res_image.transpose(), extent=high_res_extent, cmap = 'YlGnBu_r', vmin = viewer.w_vmin.value, vmax = viewer.w_vmax.value, aspect='auto'))
        if len(high_res_images) > 3:
            for im in high_res_images[:-3]:
                im.remove()
            high_res_images = high_res_images[-3:]

        
def click_echogram(event):
    global e
    e = event
    with output:
        #print(event)
        if event.button == 1:
            t = mdates.num2date(event.xdata).timestamp()
            for pn,ping in enumerate(viewer.imagebuilder.pings):
                if ping.get_timestamp() > t:
                    if pn > 0:
                        pn -= 1
                    break
                    
            print('Ping nr:', pn, 'time:', ping.get_timestamp(), 'date:', ping.get_datetime())
            viewer.w_index.value = pn
            
fig_events['on_click'] = fig_eg, fig_eg.canvas.mpl_connect("button_press_event", click_echogram)

update_button._click_handlers.callbacks = []
clear_button._click_handlers.callbacks = []
update_button.on_click(update_echogram)
clear_button.on_click(clear_output)